## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head(10)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,Saint-Philippe,RE,88.11,few clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
1,13,Nosy Varika,MG,82.60,scattered clouds,-20.5833,48.5333,Hotel Volazara
2,16,Richards Bay,ZA,80.62,overcast clouds,-28.7830,32.0377,BON Hotel Waterfront Richards Bay
3,17,Touros,BR,79.70,overcast clouds,-5.1989,-35.4608,INN NEW HORIZON
4,22,Hithadhoo,MV,81.88,scattered clouds,-0.6000,73.0833,Scoop Guest House
5,26,Jalingo,NG,86.86,few clouds,8.8833,11.3667,Debright Motel
6,28,Xuddur,SO,84.45,clear sky,4.1213,43.8894,Banjanay Hotel
7,31,Khipro,PK,82.89,clear sky,25.8285,69.3772,Mukesh Bhojraj Khatri Graphix Khipro
8,38,Mahebourg,MU,86.29,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
9,41,Georgetown,MY,89.55,scattered clouds,5.4112,100.3354,Cititel Penang


In [3]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [4]:
# 4a. Add a marker layer for each city to the map.
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df["City"] =="Olinda"]
vacation_end = vacation_df.loc[vacation_df["City"] =="Olinda"]
vacation_stop1 = vacation_df.loc[vacation_df["City"] =="Cabedelo"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] =="Natal"]
vacation_stop3 = vacation_df.loc[vacation_df["City"] =="Aquiraz"]

In [6]:
vacation_start

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
68,307,Olinda,BR,80.33,scattered clouds,-8.0089,-34.8553,Hotel Albergue de Olinda


In [7]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
import numpy as np

start = vacation_start[["Lat", "Lng"]].to_numpy()
end = vacation_end[["Lat", "Lng"]].to_numpy()
stop1 = vacation_stop1[["Lat", "Lng"]].to_numpy()
stop2 = vacation_stop2[["Lat", "Lng"]].to_numpy()
stop3 = vacation_stop3[["Lat", "Lng"]].to_numpy()

In [8]:
start[0]

array([ -8.0089, -34.8553])

In [9]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
import gmaps.datasets

fig = gmaps.figure()
itinerary = gmaps.directions_layer(start[0], end[0], waypoints=[stop1[0], stop2[0], stop3[0]], travel_mode="DRIVING")
fig.add_layer(itinerary)
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3], ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,307,Olinda,BR,80.33,scattered clouds,-8.0089,-34.8553,Hotel Albergue de Olinda
1,520,Cabedelo,BR,79.47,scattered clouds,-6.9811,-34.8339,Hotel Pigalle
2,335,Natal,BR,80.82,broken clouds,-5.7950,-35.2094,Villa Park Hotel | Natal
3,465,Aquiraz,BR,83.21,overcast clouds,-3.9014,-38.3911,Pousada Rua Camurupim


In [11]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

fig


Figure(layout=FigureLayout(height='420px'))